# Siting First Aid Stations with LSCP on Toronto's University Campus

*Authors:* [Timothy Ellersiek](https://github.com/TimMcCauley), [James Gaboardi](https://github.com/jGaboardi)

This notebook implements the location set covering problem (LSCP) to site first aid stations for Toronto's University campus. LSCP determines locations such that as many demand points as possible are within the maximum service radius. By doing so, the number of locations required to cover all demand points is minimized. In contrast to the maximum covering location problem it does not require a maximum amount of locations as an input parameter. Hence, the use cases for LSCP are specific. One common scenario is finding suitable locations for hospitals in cities where the amount is governed by reachability preconditions to households in a certain amount of time.  

In this notebook demand is described as buildings within Toronto's University campus. Possible locations for the first aid stations are determined using [Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API) which fetches geometries directly from [OpenStreetMap.org](https://openstreetmap.org).

The routing matrix is computed using [Open Source Routing Machine's demo server API](http://project-osrm.org).

## Prerequisites

To run this jupyter notebook locally, you will have to install the following additional libraries into your python environment.

- [folium](https://python-visualization.github.io/folium)
- [overpy](https://github.com/DinoTools/python-overpy)
- [routing-py](https://github.com/gis-ops/routing-py)
- [shapely](https://github.com/shapely/shapely)

In [1]:
%config InlineBackend.figure_format = "retina"
%load_ext watermark
%watermark

Last updated: 2023-12-10T13:49:32.513924-05:00

Python implementation: CPython
Python version       : 3.11.6
IPython version      : 8.18.0

Compiler    : Clang 15.0.7 
OS          : Darwin
Release     : 23.1.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



In [2]:
import folium
import json
import numpy
import overpy
import pulp
import routingpy
from routingpy import OSRM
import shapely
from shapely.geometry import shape, Point
import spopt
from spopt.locate import LSCP

%watermark -w
%watermark -iv

Watermark: 2.4.3

pulp     : 2.7.0
overpy   : 0.6
spopt    : 0.5.1.dev59+g343ef27
shapely  : 2.0.2
routingpy: 1.2.1
json     : 2.0.9
folium   : 0.15.0
numpy    : 1.26.2



--------------------------

Create `overpy.Overpass()` instance in order to make queries

In [3]:
api = overpy.Overpass()

First of all we want to fetch all buildings in the University of Toronto boundary

In [4]:
result = api.query(
    """
    [out:json];
    
    area[name="Toronto"]->.b;
    way(area.b)[name="University of Toronto"];
    map_to_area -> .a;
    
    way[building="university"](area.a);
    out center;
    """
)

Create a `folium.Map()` instance to visualize the data

In [5]:
m = folium.Map(
    location=[43.6624674, -79.3988052], tiles="cartodbpositron", zoom_start=15
)

Let's add our results to the map!

In [6]:
def get_centroid(pnt_info, xy=True):
    """Extract centroid from location information."""
    if isinstance(pnt_info, overpy.Way):
        x, y = pnt_info.center_lon, pnt_info.center_lat
        if not xy:
            x, y = y, x
    else:
        x, y = pnt_info["center"][1], pnt_info["center"][0]
    return [x, y]

In [7]:
get_tag = lambda w: w.tags.get("name", "n/a")

In [8]:
university_buildings = []
university_buildings_locations = []
university_buildings_fg = folium.FeatureGroup("University Buildings").add_to(m)
for idx, way in enumerate(result.ways):
    tag = get_tag(way)
    loc_yx = get_centroid(way, xy=False)
    folium.CircleMarker(
        location=loc_yx,
        radius=3,
        fill=True,
        fill_opacity=0.3,
        popup=folium.Popup(tag, show=False),
        color="blue",
    ).add_to(university_buildings_fg)
    # building centroids
    university_buildings.append({"name": tag, "center": get_centroid(way), "idx": idx})
    university_buildings_locations.append(loc_yx)
n_uni_bld = len(university_buildings)
m

And now we want to fetch some roads in the same area and return their center points which we can use for our potential locations for first aid stations

In [9]:
result_roads = api.query(
    """
    [out:json];
    
    area[name="Toronto"]->.b;
    way(area.b)[name="University of Toronto"];
    map_to_area -> .a;
    
    way["highway"~"secondary|tertiary|unclassified"](area.a);
    
    out center;
    """
)

Similarly we want to add them to the map

In [10]:
first_aid_stations = []
first_aid_stations_locations = []
first_aid_stations_fg = folium.FeatureGroup("First Aid Station Locations").add_to(m)
for idx, way in enumerate(result_roads.ways):
    tag = get_tag(way)
    loc_yx = get_centroid(way, xy=False)
    folium.CircleMarker(
        location=loc_yx,
        radius=2,
        fill=True,
        fill_opacity=1,
        popup=folium.Popup(tag, show=False),
        color="black",
    ).add_to(first_aid_stations_fg)
    # road midpoints
    first_aid_stations.append(
        {"name": tag, "center": get_centroid(way), "idx": idx + n_uni_bld}
    )
    first_aid_stations_locations.append(loc_yx)
m

## Computing the Distance (Routing) Matrix

Given our potential first aid station locations and the university buildings on the campus we want to compute our distance matrix. Instead of using simple euclidean distance we can make use of proper routes following roads which will yield more realistic travel times. The [routing-py library](https://github.com/gis-ops/routing-py) helps us achieve this which features the ability to use different open source frameworks such as OSRM or Valhalla as well as 3rd party API's such as Google Maps or HERE.

In [11]:
client = OSRM(base_url="https://router.project-osrm.org")

Isolate all location coordinates

In [12]:
all_locations = university_buildings_locations + first_aid_stations_locations
all_locations = [[x, y] for y, x in all_locations]
all_locations[:5]

[[Decimal('-79.3988052'), Decimal('43.6624674')],
 [Decimal('-79.3994442'), Decimal('43.6643897')],
 [Decimal('-79.3895107'), Decimal('43.6648522')],
 [Decimal('-79.3971848'), Decimal('43.6625440')],
 [Decimal('-79.3958563'), Decimal('43.6628344')]]

Define source and target indices

In [13]:
source_indices = [i["idx"] for i in university_buildings]
target_indices = [i["idx"] for i in first_aid_stations]

In [14]:
source_indices[-5:], target_indices[:5]

([117, 118, 119, 120, 121], [122, 123, 124, 125, 126])

Generate routing matrix object

In [15]:
osrm_routing_matrix = client.matrix(
    dry_run=False,
    locations=all_locations,
    sources=source_indices,
    destinations=target_indices,
    profile="pedestrian",
)

Isolate route durations

In [16]:
cost_matrix = numpy.array(osrm_routing_matrix.durations)
cost_matrix.shape

(122, 35)

In [17]:
cost_matrix

array([[242.5, 132.1, 164.3, ...,  67.5,  95.5, 135.9],
       [220.3, 109.9, 155.6, ..., 117.9,  46.9, 113.7],
       [183.5, 134.5, 152.4, ..., 285.5, 222.4, 138.3],
       ...,
       [243.1, 132.7, 178.4, ...,  97.9,  75.4, 136.5],
       [181.5,  42.6,  88.3, ..., 196.5, 234.1,  46.4],
       [136.1, 167.6, 207.2, ..., 260.6, 211.2, 171.4]])

## Solving the LSCP

Last but not least we want to run the solver. We choose the `pulp.COIN_CMD` and set the distance cutoff to 2 minutes and 30 seconds afoot. Finally we loop over the solution and add the sited stations and the corresponding allocated buildings to the map with different radiuses.  

In [18]:
# maximum acceptable service duration, 2 minutes and 30 seconds
SERVICE_RADIUS = 150

# set the solver (for available solvers run ``pulp.listSolvers(onlyAvailable=True)``)
solver = pulp.COIN_CMD(msg=False)

Create and solve an LSCP instance

In [19]:
lscp = LSCP.from_cost_matrix(cost_matrix, SERVICE_RADIUS)
lscp = lscp.solve(solver)

How many first aid sites are needed to entirely cover the university buildings?

In [20]:
lscp.problem.objective.value()

3.0

Add the results to the map

In [21]:
sited_first_aid_stations_fg = folium.FeatureGroup("Sited First Aid Stations").add_to(m)
allocated_buildings_demand_fg = folium.FeatureGroup("Allocated Buildings").add_to(m)
solution = {}
colors = [
    "darkcyan",
    "mediumseagreen",
    "saddlebrown",
    "lightskyblue",
    "darkgoldenrod",
    "coral",
    "mediumvioletred",
    "blueviolet",
    "mediumorchid",
]
client_marker_size = 20
numpy.random.seed(0)
for i in range(len(first_aid_stations)):
    if lscp.fac2cli[i]:
        color_choice_idx = numpy.random.choice(range(len(colors)))
        color = colors.pop(color_choice_idx)

        # selected candidate facility sites
        folium.CircleMarker(
            location=first_aid_stations_locations[i],
            radius=30,
            fill=True,
            popup=folium.Popup(f"Sited First Aid Station (index: {i})", show=True),
            color=color,
        ).add_to(sited_first_aid_stations_fg)

        # client locations with associated facility symbology
        client_marker_size -= 4
        solution[i] = []
        for j in lscp.fac2cli[i]:
            solution[i].append(university_buildings[j])
            folium.CircleMarker(
                location=university_buildings_locations[j],
                radius=client_marker_size,
                fill=True,
                popup=f"Building allocated to First Aid Station with index: {i}",
                color=color,
            ).add_to(allocated_buildings_demand_fg)
folium.LayerControl().add_to(m)
m

It is ***important to note*** that the modeling solution is a direct result of the input cost matrix (seconds of travel) obtained from `routingpy.OSRM`. Although the solution set and decisions may not appear to optimal, the formulation of and solution to the LSCP are correct based on the input matrix.

In [22]:
solution

{6: [{'name': 'Sidney Smith Hall',
   'center': [Decimal('-79.3988052'), Decimal('43.6624674')],
   'idx': 0},
  {'name': 'Robarts Library',
   'center': [Decimal('-79.3994442'), Decimal('43.6643897')],
   'idx': 1},
  {'name': 'Koffler Student Services Centre',
   'center': [Decimal('-79.3968996'), Decimal('43.6591667')],
   'idx': 6},
  {'name': 'Daniels Building',
   'center': [Decimal('-79.4007115'), Decimal('43.6596491')],
   'idx': 7},
  {'name': 'Warren Stevens Building',
   'center': [Decimal('-79.4010325'), Decimal('43.6627537')],
   'idx': 8},
  {'name': 'Ramsay Wright Building',
   'center': [Decimal('-79.3990600'), Decimal('43.6632684')],
   'idx': 13},
  {'name': 'Clara Benson Building',
   'center': [Decimal('-79.4002584'), Decimal('43.6629833')],
   'idx': 14},
  {'name': 'Fisher Rare Book Library',
   'center': [Decimal('-79.3989770'), Decimal('43.6640748')],
   'idx': 15},
  {'name': 'Simcoe Hall',
   'center': [Decimal('-79.3959493'), Decimal('43.6607473')],
   'idx':

--------------------